In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Установите память GPU в режиме роста, чтобы не использовать всю память сразу
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, using CPU")

img_height, img_width = 224, 224
batch_size = 32
epochs = 40
learning_rate = 0.001

base_dir = './'
train_dir = base_dir

class_labels = {
    'alcohol': 0,
    'bloody_scenes': 1,
    'drugs': 2,
    'porn': 3,
    'smoking': 4,
    'safe_content': 5
}

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=list(class_labels.keys()),
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=list(class_labels.keys()),
    subset='validation'
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

import time
model.save(f'model_{round(time.time())}.keras')


: 

In [23]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_33 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 133,197,524 (508.11 MB)

 Trainable params: 44,399,174 (169.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 88,798,350 (338.74 MB)

In [26]:
import os
def classify_image(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0  # Добавляем batch размер и нормализуем

    predictions = model.predict(img_array)
    predicted_class = tf.argmax(predictions[0]).numpy()
    print(image_path, predictions)
    return list(class_labels.keys())[predicted_class]

image_path = '.\\bloody_scenes'
with open('res.txt', 'w') as f:
    for file in os.listdir(image_path):
        f.write(file + ' is ' + classify_image(os.path.join(image_path, file), model) + '\n')
print('saved')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
.\bloody_scenes\bloody_scenes_0.jpg [[9.58711743e-01 6.89684384e-05 1.21574955e-08 4.12004068e-02
  1.88776849e-05 3.28979439e-08]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
.\bloody_scenes\bloody_scenes_10.jpg [[9.9999642e-01 1.9305089e-06 5.8140064e-14 1.5690890e-11 1.6736215e-06
  3.9449242e-14]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
.\bloody_scenes\bloody_scenes_11.jpg [[4.5426045e-02 2.5664270e-03 1.9851165e-04 1.0466498e-02 9.4090980e-01
  4.3276933e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
.\bloody_scenes\bloody_scenes_12.jpg [[9.1887850e-01 1.4335284e-03 4.1292171e-08 6.5218724e-02 8.4710649e-05
  1.4384572e-02]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
.\bloody_scenes\bloody_scenes_13.jpg [[8.5522860e-01 9.7524165e-04 3.1276617e-09 3.0220917e-04 1.4343722e-01
  5.6771234e-05]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
.\bloody_scenes\bloody_scenes_14.jpg [[1.40661765e-02 9.84556675e-01 7.06388846e-07 3.82227445e-04
  2.78020510e-04 7.16176408e-04]]